#### Prepare Cellpose training dataset  

30 training images from the HCC1143 series of experiments. Early and late Untreated plus late high dose from each drug. For nuclear segmentation, apply the red channel as a mask to the phase, then combine them to get the red channel "textured" by the phase images. Use red and phase as separate channels for cytoplasmic segmentation.  Use registered intensity images and prefiltered masks. Masks must have the same name as the intensity images with a suffix added before the file type. 

In [23]:
import numpy as np
import numpy.ma as ma
import os, glob, tifffile
from skimage import io, filters, exposure

In [24]:
def transfer_r_and_p_file(data_path, cellline, plateID, well, field_num, time_slice):
    
    #get the cropped r image
    filename = os.path.join(data_path, plateID,"Analysis","registered_stacks",plateID+"_R_"+well+"_"+str(field_num)+"_reg_stack.tif")
    r_image = io.imread(filename)[int(time_slice)]
    
    #get the cropped p image
    filename = os.path.join(data_path, plateID,"Analysis","registered_stacks",plateID+"_P_"+well+"_"+str(field_num)+"_reg_stack.tif")
    p_image = io.imread(filename)[int(time_slice)]
    
    r_and_p_image = np.stack((r_image, p_image))
    #add the cropped r image to the training set of images
    out_filename = os.path.join(data_path,"cellpose_Ctc_"+cellline, "images",plateID+"_"+well+"_"+str(field_num)+"_"+time_slice+"_r_and_p_img.tif")
    print("adding "+out_filename+" to image set")
    tifffile.imwrite(out_filename, r_and_p_image)
    print("adding "+out_filename.replace("_r_and","")+" to image set")
    tifffile.imwrite(out_filename.replace("_r_and",""), p_image)

def transfer_p_file(input_data_path, output_data_path, cellline, plateID, well, field_num, time_slice):
    
    #get the cropped p image
    filename = os.path.join(input_data_path, plateID,"Analysis","registered_stacks",plateID+"_P_"+well+"_"+str(field_num)+"_reg_stack.tif")
    p_image = io.imread(filename)[int(time_slice)]
    
    #add the cropped p image to the training set of images
    out_filename = os.path.join(output_data_path,"cellpose_Ccnt_"+cellline, "images",plateID+"_"+well+"_"+str(field_num)+"_"+time_slice+"_p_img.tif")
    print("adding "+out_filename+" to image set")
    tifffile.imwrite(out_filename, p_image)
    
def transfer_R_file(input_data_path, output_data_path, cellline, plateID, well, field_num, time_slice):
    
    #get the cropped R image
    filename = os.path.join(input_data_path, plateID,"Analysis","registered_stacks",plateID+"_R_"+well+"_"+str(field_num)+"_reg_stack.tif")
    R_image = io.imread(filename)[int(time_slice)]
    
    #add the cropped R image to the training set of nuclei images
    out_filename = os.path.join(output_data_path,"cellpose_Ccnt_"+cellline, "images_Ctn",plateID+"_"+well+"_"+str(field_num)+"_"+time_slice+"_R_img.tif")
    print("adding "+out_filename+" to image set")
    tifffile.imwrite(out_filename, R_image)

hist, bins_center = exposure.histogram(image)

plt.figure(figsize=(18, 8))
plt.subplot(131)
plt.imshow(image, cmap='gray', interpolation='nearest')
plt.axis('off')
plt.subplot(132)
plt.imshow(nuc_mask, cmap='gray', interpolation='nearest')
plt.axis('off')
plt.subplot(133)
plt.imshow(phase_image_masked, cmap = 'gray', interpolation = 'nearest')
plt.tight_layout()

In [27]:
input_data_path = "/home/exacloud/gscratch/HeiserLab/images/"
output_data_path = "/home/groups/heiserlab_genomics/home/dane/CellTracking/images/"
plateID_list = ("MD00301", "MD00401")
#plateID_list = (["MD00301"])
plateID_list = ("HC01601", "HC01801", "HC02001")
#plateID_list = (["HC01501"])

cellline = "MDAMB157"
cellline = "21MT1"
cellline = "HCC1143nlc_20x"
#cellline = "XX"

model_name = "Ctc"
well_list = ("A1", "A2","A3", "A4", "A5", "A6",
            "B1", "B2","B3", "B4", "B5", "B6",
            "C1", "C2","C3", "C4", "C5", "C6",
            "D1", "D2","D3", "D4", "D5", "D6")

#well_list = (["A1"])
well_list = ("B2","C2")
time_slice_list = ("004", "100","189")
well_list = (["B2", "B2", "B2"])
field_num = 1


In [29]:
res = list(map(transfer_p_file,
               [input_data_path]*len(well_list)*len(plateID_list),
               [output_data_path]*len(well_list)*len(plateID_list),
              [cellline]*len(well_list)*len(plateID_list),
               [element for element in plateID_list for i in range(len(well_list))],
               well_list*len(plateID_list),
               [field_num]*len(well_list)*len(plateID_list),
               time_slice_list*len(plateID_list)))

res = list(map(transfer_R_file,
               [input_data_path]*len(well_list)*len(plateID_list),
               [output_data_path]*len(well_list)*len(plateID_list),
              [cellline]*len(well_list)*len(plateID_list),
               [element for element in plateID_list for i in range(len(well_list))],
               well_list*len(plateID_list),
               [field_num]*len(well_list)*len(plateID_list),
               time_slice_list*len(plateID_list)))


adding /home/groups/heiserlab_genomics/home/dane/CellTracking/images/cellpose_Ccnt_HCC1143nlc_20x/images/HC01601_B2_1_004_p_img.tif to image set
adding /home/groups/heiserlab_genomics/home/dane/CellTracking/images/cellpose_Ccnt_HCC1143nlc_20x/images/HC01601_B2_1_100_p_img.tif to image set
adding /home/groups/heiserlab_genomics/home/dane/CellTracking/images/cellpose_Ccnt_HCC1143nlc_20x/images/HC01601_B2_1_189_p_img.tif to image set
adding /home/groups/heiserlab_genomics/home/dane/CellTracking/images/cellpose_Ccnt_HCC1143nlc_20x/images/HC01801_B2_1_004_p_img.tif to image set
adding /home/groups/heiserlab_genomics/home/dane/CellTracking/images/cellpose_Ccnt_HCC1143nlc_20x/images/HC01801_B2_1_100_p_img.tif to image set
adding /home/groups/heiserlab_genomics/home/dane/CellTracking/images/cellpose_Ccnt_HCC1143nlc_20x/images/HC01801_B2_1_189_p_img.tif to image set
adding /home/groups/heiserlab_genomics/home/dane/CellTracking/images/cellpose_Ccnt_HCC1143nlc_20x/images/HC02001_B2_1_004_p_img.ti

FileNotFoundError: [Errno 2] No such file or directory: '/home/groups/heiserlab_genomics/home/dane/CellTracking/images/cellpose_Ccnt_HCC1143nlc_20x/images_Ctn/HC01601_B2_1_004_R_img.tif'